# __IE 535 Computing Project - Linear Program Solver by Two-Phase Simplex Algorithm__ 
**Student Details:**
<br>
**Name:**   Kaustubh Rajimwale
<br>
**PUID:**   0033746187

<br>

## Contents:

 - Functions and Code
 - Model 24 [* Marked]
 - Model 24 - Linprog Solved
 - Model 12
 - Model 12 - Linprog Solved


**________________________________________________________________________________________________________________________________________**

### **This cell imports 4 libraries:**
[NumPy](https://numpy.org/) -  To help us develop arrays and matrices for large LP(s).
<br>
[inv function](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) - Compute the (multiplicative) inverse of a matrix.
<br>
[genfromtxt](https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html) - To call matrices A, b and C from .csv files.
<br>
[tabulate](https://pypi.org/project/tabulate/) - To print a readable tableau.

In [214]:
import numpy as np
from numpy.linalg import inv
from numpy import genfromtxt
from tabulate import tabulate

### **Function - header_names:**
This functions prints the header row of a tableau, i.e: x1 , x2 , x3 , ... RHS.

In [215]:
def header_names(tableau):
    #function takes input of the Tableau to find the size and then generate a header column from X1 to Xn
    m,n = np.shape(tableau)
    n = n-1
    headers = {}
    for i in range(n):
        headers[ "x" + str( i+1 ) ] = i
        i+=1
    #the last column is RHS, with the objective function and the BFS values
    headers["RHS"] = n
    return headers

<br>

### **Function - standard_input:**
This function uses the genfromtxt to import the matrices from .csv files.
<br>

| 0 | 1 | 1 | -1 | 0 | 0
| ---: | ---: | ---: | ---: | ---: | ---:
|  | -1 | 1 | 0 | -1 | 0
|  | 0 | 1 | 0 | 0 | 1

The format of the CSV file is given here. The 0 at the start must not be removed while entering the Matrix.

<br>

In [216]:

def standard_input():
    # This function uses the genfromtxt to import the matrices from .csv files.
    A = genfromtxt('/Users/kaustubhrajimwale/Desktop/Fall 21/IE535/Computing Project/Model24/A.csv', delimiter=',')
    A = np.delete(A, 0, 1)

    b = genfromtxt('/Users/kaustubhrajimwale/Desktop/Fall 21/IE535/Computing Project/Model24/b.csv', delimiter=',')
    b = np.delete(b, 0, 1)

    C = genfromtxt('/Users/kaustubhrajimwale/Desktop/Fall 21/IE535/Computing Project/Model24/C.csv', delimiter=',')
    C = np.delete(C, 0, 1)

    z = input("\nMaximize(1) or Minimize(0): ")
    z = int(z)

    #if maximize the C is muliplied by -1, to convert to mimimization problem
    if z != 0:
        C = C * (-1)
    
    return(A,b,C,z)

### **Function - add_artificial_variables:**
This function adds artificial variables in all equations and also finds the phase 1 cost function.

In [217]:

def add_artifical_variables(A):
    # the function takes the input of A matrix and modifies it to add the artificial variables
    # it also returns the phase 1 
    m,n = np.shape(A)
    C_phase1 = np.zeros(((n+m),1))
    for i in range(m):
        x_a = np.zeros((m,1))
        x_a[i,0] = 1
        A = np.concatenate((A, x_a), 1)
        C_phase1[n+i,0] = 1
    return A, C_phase1

### **Function - initial_basic_variables:**
This function finds the starting basic variables from the A matrix. This are the ones with the columns of an identity matrix from the RHS.

In [218]:
def initial_basic_variables(A):
    #This function finds the starting basic variables from the A matrix.
    #This are the ones with the columns of an identity matrix from the RHS.
    m,n = np.shape(A)
    l = 0
    k = 0
    Xb = np.zeros((m))
    for l in range(m):
        temp = np.zeros((m))
        temp[l] = 1
        for k in range(n):
            if np.all(A[:,n-k-1]==temp):
                Xb[l] = n-k
                break
            k+=1
        l+=1
    #returns the vector of Xb which includes variables which are in the basis
    return Xb

### **Function - basic_matrix:**
This function finds the B matrix from A and Xb

In [219]:
def basic_matrix(A,Xb):
    m,n = np.shape(A)
    B = np.zeros((m, m))
    i = 0
    j = 0
    for i in range(m):
        for j in range(m):
            B[i,j] = A[i, int(Xb[j])-1]
    #this returns the basic matrix, i.e: the columns of A for the variables are in the basis
    return B

### **Function - BFS:**
Computing the BFS which is B^(-1)*b.

In [220]:
def BFS(B,b):
    #Computing the BFS which is B^(-1)*b.
    Binv = inv(B)
    Binv_b = Binv @ b
    return Binv_b, Binv

### **Function - find_Cb:**
Finding the cost coefficients of the basic variables(Xb).

In [221]:
def find_Cb(C, Xb,A):
    #Finding the cost coefficients of the basic variables(Xb).
    m,n = np.shape(A)
    Cb = np.zeros((m,1))
    i = 0
    for i in range(m):
        Cb[i] = (C[int(Xb[i]-1)])
    return Cb

### **Function - initiate_tableau:**
Starting the tableau, and defining the values of all rows except 0th row.

In [222]:
def initiate_tableau(A, Binv_b):
    #Starting the tableau, and defining the values of all rows except 0th row.
    m,n = np.shape(A)
    tableau = np.zeros((m+1,n+1))
    tableau[1:m+1, 0:n] = A
    tableau[1:m+1, n] = np.asarray(Binv_b[:,0])
    return tableau


### **Function - reduced_cost:**
This function takes in the tableau and updates the 0th row of reduced costs.

In [223]:

def reduced_cost(Cb,A,C,tableau):
    #This function takes in the tableau and updates the 0th row of reduced costs.
    m,n = np.shape(A)
    tableau[0,0:n] = np.subtract((np.transpose(Cb) @ A), np.transpose(C))
    return tableau


### **Function - obj**
This function updates the last column of 0th row in tableau to Cb*Binv*b (objective function value)

In [224]:

def obj(tableau,Cb,A):
    #updates the objective function value in the tableau
    m,n = np.shape(A)
    tableau[0,n] = (np.transpose(Cb) @ tableau[1:m+1, n])
    return tableau

### **Function - initiate**
This function is a combination of all above function required to starting the tableau. It then also prints the starting tableau (Iteration 0).

In [225]:

def initiate():
    #This function is a combination of all above function required to starting the tableau. It then also prints the starting tableau (Iteration 0).
    A,b, C, z = standard_input()
    m,n = np.shape(A)
    A, C_phase1 = add_artifical_variables(A)
    Xb = initial_basic_variables(A)
    B = basic_matrix(A,Xb)
    Binv_b, Binv = BFS(B, b)
    Cb = find_Cb(C_phase1,Xb,A)
    tableau = initiate_tableau(A, Binv_b)
    tableau = reduced_cost(Cb,A,C_phase1,tableau)
    tableau = obj(tableau,Cb,A)
    return (tableau,Xb,C,z)

### **Function - simplex_iterations**
This function is the most significant part of the algorithm. This takes in the input of tableau and basic variables found from the initiate function and does the simplex iterations until a terminating condition is reached.

In [226]:

def simplex_iterations(tableau,Xb):

    #This function is the most significant part of the algorithm.
    #This takes in the input of tableau and basic variables found from the initiate function,
    #and does the simplex iterations until a terminating condition is reached.

    ### PRINTER
    print("\n===============================")
    print("STARTING TABLEAU")
    print("===============================\n")
    print(tabulate(np.round(tableau,10), tablefmt="github", headers=header_names(tableau), numalign="right"))
    #PRINTER

    #initializing the variables needed for the iterations
    m,n = np.shape(tableau)
    m = m-1                 #number of constraints = m
    n = n-1                 #number of variables = n
    iter = 0                #number of iteration
    flag = 1                #flag is a condition variable which shows if we should continue or terminate iterations
    optimal = 0             #an optimal tableau will have this variable = 1 
    unbdd = 0               #an unbounded tableau will have this variable = 1
    pivot_col = int(0)      #pivot_col is the tableau column index of the entering variable in any iteration. (default = 0)


    #iterate until a terminating condition is reached
    while(flag != 0):       #flag is equal to 0 when further iterations are not possible.
        
        #printing the current tableau
        iter += 1           #the number of iterations are appended by 1
        print("\n===============================")
        print("ITERATION ",iter)
        print("===============================\n")
        print(tabulate(np.round(tableau,10), tablefmt="github", headers=header_names(tableau),numalign="right"),"\n\nBasic Variables:",Xb)


        if (np.all(np.round(tableau[0,0:n],10)<=0)):        #if reduced cost for all variables is <= 0
            flag = 0                                        #The current BFS is optimatl and break the iterations loop
            flag = int(flag)                                #to stop flag = 0, and loop break
            optimal = 1                                     #optimal variable = 1
            break

        else:
            maxRC = 0                                       #this variable stores the value of maximum Reduced Cost
            i=0
            for i in range(n):                              #iterating through all reduced cost to find maximum reduced cost
                if tableau[0,i]>=maxRC:
                    maxRC = tableau[0,i]                    #if any reduced cost is greater than previously found RC, then update RC
                    pivot_col = i                           #pivot column (entering variable) index is stored
                i+=1
            
            y = (tableau[1:m+1,pivot_col]<=0)               #the values of all the entries the column of entering variables
            if (sum(y)==m):                                 #if all the yi<= 0 then the LP is unbounded, STOP!
                flag = 0
                flag = int(flag)
                unbdd = 1
                break                                       #unbounded = 1 and STOP the iterations.
            


            #ratio_test and anticycling rule:

            ratio = 10000000001                             #'ratio' stores the value of the min(b/y)i for y>0 (initially = large number)
            i=0
            for i in range(m):                              #now starting from the first row (first constraint)
                #if the column entry of entering variable of ith row > 0 (y>0); and (b/y<=existing ratio):
                if (tableau[i+1,pivot_col]>0) and ((tableau[i+1,n]/tableau[i+1,pivot_col])<=ratio):         
                    if (tableau[i+1,n]/tableau[i+1,pivot_col]) < ratio: #if the ratio is less than the existing ratio, then:
                        ratio = (tableau[i+1,n]/tableau[i+1,pivot_col]) #update the ratio variable to b/y
                        pivot_row = i+1                                 #the pivot_row = index of leaving variable is update to i+1
                    else:                                               #in this case the ratio of (bi/yi) is equal to exisitng ratio
                        #anticycling rule is needed - here I have used Lexicographic approach
                        lexicographic = tableau[i+1,:] - tableau[pivot_row,:]       #the lexicographic variable takes the u - v
                        #the difference i.e: u-v is between the two rows with same b/y ratio
                        #to proceed we have to take the lexicographically greater row for the leaving variable
                        j=0
                        while(i>=0):
                            if lexicographic[j]>0:          #if the first non-zero element of u-v is positive, u is lexicographically larger
                                ratio = (tableau[i+1,n]/tableau[i+1,pivot_col])  # update ratio to bi/yi since ith col is lexi. larger than existing 
                                pivot_row = i+1             #new leaving variable
                                break                       #stop search for any more non-zero elements in u-v
                            else:
                                if lexicographic[j]<0:      #this means the first non-zero element of u-v is negative, this means u is smaller
                                    break                   #no updating required, break. The current leaving variable is 
                                else:                       #if the current(jth element) of u-v = 0
                                    j+=1                    #see if the next elemet is non-zero
                i+=1
            print("\nPivot Element:",tableau[pivot_row,pivot_col],"\n")
            print("Entering Variable: x",pivot_col+1)
            print("Leaving Variable: x",int(Xb[pivot_row-1]),"\n")
            Xb[pivot_row-1] = pivot_col+1



            #Row Transformations
            i=0
            tableau[pivot_row,:] = tableau[pivot_row,:]/tableau[pivot_row,pivot_col]
            for i in range(m+1):
                if i!=pivot_row:
                    tableau[i,:] = tableau[i,:] - (tableau[pivot_row,:]*(tableau[i,pivot_col]))
                i+=1         

                    
    return(tableau,unbdd,optimal,pivot_col, Xb)

In [227]:
def result_phase1(unbdd, optimal, tableau, Xb, C):
    m,n = np.shape(tableau)
    m = m-1
    n = n-1
    phase2 = 1

    logic = Xb > n-m

    if(sum(logic)!=0):
        count = 0
        i = 0
        artificial_variables_in_basis =  np.zeros((0))
        temp = np.zeros((1))
        temp_row = np.zeros((1))
        rows_to_remove = np.zeros((0))
        val = np.zeros((0))

        for i in range(m):
            if Xb[i] > n-m:
                temp[0] = Xb[i]
                artificial_variables_in_basis = np.append(artificial_variables_in_basis,temp,axis= 0)
                temp[0] = round((tableau[i+1,n]),10)
                val = np.append(val,temp,axis= 0)
                if temp == 0:
                    temp_row[0] = i+1
                    rows_to_remove = np.append(rows_to_remove,temp_row,axis= 0)
            i+=1

        if (sum(val == 0)!=0):
            print("\nRedundancy Exists! Removing the redundant constraint.\n")
            l, = np.shape(rows_to_remove)
            i = 0
            print("...")
            for i in range(l):
                tableau = np.delete(tableau, int(rows_to_remove[i]), 0)
                Xb = np.delete(Xb, int(rows_to_remove[i]-1), 0)
                print("...")
                i+=1
            print("\nRedundant Row Deleted\n")

        else:
            print("\nThe linear program is infeasible, as the basic variables of the final tableau consists of one or more artificial variables with non-zero value.\nBasic Variables:")
            for i in range(m):
                print("X",int(Xb[i]),"=",(tableau[i+1,n]))
            print(" ")
            phase2 = 0
            return 0,0,0

    
    else:
        print("\nThe given LP is feasible. Starting Phase II now.\n")
    
    if phase2 == 1:
        i = 0
        for i in range(m):
            tableau = np.delete(tableau, n-i-1, 1)
            i+=1
    
        m,n = np.shape(tableau)
        m = m-1
        n = n-1    
        A = np.zeros((m,n))
        Cb = find_Cb(C,Xb,A)
        zero_col = np.array([[0]])
        temp = np.append(np.transpose(C), zero_col, 1)
        tableau[0,0:n+1] = np.subtract((np.transpose(Cb) @ tableau[1:m+1,:]), temp)
        print("Updated Tableau:\n")
        print(tabulate(np.round(tableau,10), tablefmt="github", headers=header_names(tableau), numalign="right"),"\n\nBasic Variables:",Xb)
        print(" ")
        print("\n End of Phase I. \n Phase II:\n")
        return tableau, Xb, 1

In [228]:

def result_phase2(unbdd,optimal,tableau,Xb,z):
    m,n = np.shape(tableau)
    m = m-1
    n = n-1
    if optimal == 1:
        print(" ")
        print("ITERATIONS STOPPED\n")
        print("The current BFS is optimal:\n")
        if z != 0:
            print("Objective Function =",(tableau[0,n]*(-1)))
        else:
            print("Objective Function =",(tableau[0,n]*(1)))
        j=0
        for j in range(m):
            print("X",int(Xb[j]),"=",(tableau[j+1,n]))
        print(" ")
        print("All other variables are non-basic and are equal to 0.")
        print(" ")
        print(" ")

    if unbdd == 1:
        print("THE LP IS UNBOUNDED ALONG THE DIRECTION:")
        dir = np.zeros((n,1))
        x_unbdd = np.zeros((n,1))
        i = 0
        for i in range(m):
            x_unbdd[int(Xb[i]-1),0] = tableau[i+1,n]
            dir[int(Xb[i]-1),0] = -1*tableau[i+1,pivot_col]
        dir[pivot_col,0] = 1
        i = 0
        print(" ")
        for i in range(n):
            print('x',i+1,' = ',x_unbdd[i,0],"+",dir[i,0],'T')
        print(" ")


In [229]:

tableau,Xb,C, z = initiate()
tableau,unbdd,optimal,pivot_col, Xb = simplex_iterations(tableau,Xb)
tableau, Xb, next_phase = result_phase1(unbdd, optimal, tableau, Xb,C)
if next_phase == 1:
    tableau,unbdd,optimal,pivot_col, Xb = simplex_iterations(tableau,Xb)
    result_phase2(unbdd, optimal, tableau, Xb, z)


STARTING TABLEAU

|    x1 |    x2 |    x3 |    x4 |    x5 |    x6 |    x7 |    x8 |    x9 |   x10 |   x11 |   x12 |   x13 |   x14 |   x15 |   x16 |   x17 |   x18 |   x19 |   x20 |   x21 |   x22 |   x23 |   x24 |   x25 |   x26 |   x27 |   x28 |   x29 |   x30 |   x31 |   x32 |   x33 |   x34 |   x35 |   x36 |   RHS |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
|  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |  1.55 |     1 |     1 |     1 |     1 |     1 |     1 |     1 |     1 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 17400 |
|     1 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [230]:
import scipy
from scipy.optimize import linprog
A,b,C,z = standard_input()
scipy.optimize.linprog(C, A_eq=A, b_eq=b, method='simplex')

     con: array([ 0.00000000e+00, -1.13686838e-13,  0.00000000e+00,  2.27373675e-13,
        9.09494702e-13, -1.36424205e-12, -4.54747351e-13,  0.00000000e+00,
        1.06936682e-12, -2.27373675e-13, -3.41060513e-13, -6.25277607e-13,
        5.68434189e-14,  0.00000000e+00])
     fun: 5014.363636363636
 message: 'Optimization terminated successfully.'
     nit: 23
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 890.90909091,  309.09090909,  542.72727273,  357.27272727,
          0.        ,    0.        , 1700.        ,  148.18181818,
       1851.81818182,    0.        , 2500.        ,    0.        ,
          0.        ,  709.09090909,    0.        ,   90.90909091,
          0.        , 1435.90909091, 1000.        ,  395.        ,
          0.        ,  904.09090909])

MODEL 12

In [231]:

def standard_input_model12():
    A = genfromtxt('/Users/kaustubhrajimwale/Desktop/Fall 21/IE535/Computing Project/Model12/A.csv', delimiter=',')
    A = np.delete(A, 0, 1)

    b = genfromtxt('/Users/kaustubhrajimwale/Desktop/Fall 21/IE535/Computing Project/Model12/b.csv', delimiter=',')
    b = np.delete(b, 0, 1)

    C = genfromtxt('/Users/kaustubhrajimwale/Desktop/Fall 21/IE535/Computing Project/Model12/C.csv', delimiter=',')
    C = np.delete(C, 0, 1)

    z = input("\nMaximize(1) or Minimize(0): ")
    z = int(z)

    if z != 0:
        C = C * (-1)
    
    return(A,b,C,z)

In [232]:

def initiate_model12():
    A,b, C, z = standard_input_model12()
    m,n = np.shape(A)
    A, C_phase1 = add_artifical_variables(A)
    Xb = initial_basic_variables(A)
    B = basic_matrix(A,Xb)
    Binv_b, Binv = BFS(B, b)
    Cb = find_Cb(C_phase1,Xb,A)
    tableau = initiate_tableau(A, Binv_b)
    tableau = reduced_cost(Cb,A,C_phase1,tableau)
    tableau = obj(tableau,Cb,A)
    return (tableau,Xb,C,z)

In [233]:
tableau,Xb,C, z = initiate_model12()
tableau,unbdd,optimal,pivot_col, Xb = simplex_iterations(tableau,Xb)
tableau, Xb, next_phase = result_phase1(unbdd, optimal, tableau, Xb,C)
if next_phase == 1:
    tableau,unbdd,optimal,pivot_col, Xb = simplex_iterations(tableau,Xb)
    result_phase2(unbdd, optimal, tableau, Xb, z)


STARTING TABLEAU

|   x1 |   x2 |   x3 |   x4 |   x5 |   x6 |   x7 |   x8 |   x9 |   RHS |
|------|------|------|------|------|------|------|------|------|-------|
|  101 |  101 |  101 |  101 |  101 |    0 |    0 |    0 |    0 |   101 |
|   60 |   25 |   45 |   20 |   50 |    1 |    0 |    0 |    0 |    40 |
|   10 |   15 |   45 |   50 |   40 |    0 |    1 |    0 |    0 |    35 |
|   30 |   60 |   10 |   30 |   10 |    0 |    0 |    1 |    0 |    25 |
|    1 |    1 |    1 |    1 |    1 |    0 |    0 |    0 |    1 |     1 |

ITERATION  1

|   x1 |   x2 |   x3 |   x4 |   x5 |   x6 |   x7 |   x8 |   x9 |   RHS |
|------|------|------|------|------|------|------|------|------|-------|
|  101 |  101 |  101 |  101 |  101 |    0 |    0 |    0 |    0 |   101 |
|   60 |   25 |   45 |   20 |   50 |    1 |    0 |    0 |    0 |    40 |
|   10 |   15 |   45 |   50 |   40 |    0 |    1 |    0 |    0 |    35 |
|   30 |   60 |   10 |   30 |   10 |    0 |    0 |    1 |    0 |    25 |
|    1 |    1 |  

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [234]:
import scipy
from scipy.optimize import linprog
A,b,C,z = standard_input_model12()
scipy.optimize.linprog(C, A_eq=A, b_eq=b, method='simplex')

<ipython-input-234-36283aada42e>:4: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  scipy.optimize.linprog(C, A_eq=A, b_eq=b, method='simplex')


     con: array([-7.10542736e-15,  7.10542736e-15,  7.10542736e-15,  0.00000000e+00])
     fun: 23.456521739130434
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([0.04347826, 0.2826087 , 0.67391304, 0.        , 0.        ])